## MLflow Configuration

In [ ]:
# Installation de librairie 
!pip install mlflow
!pip install --upgrade jinja2
!pip install --upgrade Flask
!pip install setuptools

In [ ]:
import pandas as pd
import numpy as np
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Configuration de mon Mlflow sur un server local 
# à exéuter dans le terminal
#!mlflow server --host 127.0.0.1 --port 8080

In [17]:
## Connection au port 
ML= MlflowClient(tracking_uri="http://127.0.0.1:8080")

In [18]:
#Suivit des expériments
all_experiments = ML.search_experiments()
print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1760395304471, experiment_id='0', last_update_time=1760395304471, lifecycle_stage='active', name='Default', tags={}>]


## Random Forest model

In [19]:
ML.search_experiments(filter_string="name = 'RDF_Models'")

[]

In [20]:
# Import de la donnée
data_dir = r"C:\Users\j_aka\Desktop\mlops\credit_risk_dataset.csv"
df = pd.read_csv(data_dir, delimiter=",") 
print(df.head(3).to_string())

   person_age  person_income person_home_ownership  person_emp_length loan_intent loan_grade  loan_amnt  loan_int_rate  loan_status  loan_percent_income cb_person_default_on_file  cb_person_cred_hist_length
0          22          59000                  RENT              123.0    PERSONAL          D      35000          16.02            1                 0.59                         Y                           3
1          21           9600                   OWN                5.0   EDUCATION          B       1000          11.14            0                 0.10                         N                           2
2          25           9600              MORTGAGE                1.0     MEDICAL          C       5500          12.87            1                 0.57                         N                           3


In [21]:
## Import de librairie
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [22]:
## Connection Mlflow 
# This function call sets the global tracking URI for the current session.
# It’s a convenient way to configure the tracking server URI without creating a separate client instance.

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [24]:
# Sets the current active experiment to the "Apple_Models" experiment and
# returns the Experiment metadata
RDF_experiment = mlflow.set_experiment("RDF_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "RDF_rf_test"

# Define an artifact path that the model will be saved to.
artifact_path = "RDF_apples"

2025/10/14 00:57:54 INFO mlflow.tracking.fluent: Experiment with name 'RDF_Models' does not exist. Creating a new experiment.


In [26]:
# Split the data into features and target and drop irrelevant date field and target field
#X = df.drop(columns=["loan_percent_income", "loan_int_rate","loan_grade"])
X = df[["loan_percent_income", "loan_int_rate", "loan_grade"]]
y = df["loan_status"]

grade_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
X["loan_grade"] = X["loan_grade"].map(grade_map)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestclassifier
rf = RandomForestClassifier(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(
        sk_model=rf, input_example=X_val, artifact_path=artifact_path
    )


C:\Users\j_aka\AppData\Local\Temp\ipykernel_36128\345315044.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["loan_grade"] = X["loan_grade"].map(grade_map)
2025/10/14 00:58:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\j_aka\anaconda3\envs\statistique\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values.

🏃 View run RDF_rf_test at: http://127.0.0.1:8080/#/experiments/137361489074130957/runs/78867c2977794726974ec4e819575dc5
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/137361489074130957
